In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/punjabi-emotion-data/validation_data.csv
/kaggle/input/punjabi-emotion-data/training_data.csv
/kaggle/input/punjabi-emotion-data/test_data.csv


In [3]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

In [4]:
from datasets import load_dataset
dataset = load_dataset('emotion')

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:

# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [6]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
train_dataset = pd.read_csv('/kaggle/input/punjabi-emotion-data/training_data.csv')
train_dataset.head()


# train_dataset = dataset['train']
text_train=[]
label_train=[]
for i in range(len(train_dataset)):
    text_train.append(train_dataset.iloc[i]['punjabi_emotion'])
    label_train.append(train_dataset.iloc[i]['labels'])
len(text_train),len(label_train)

train_df = pd.DataFrame(list(zip(text_train, label_train)),
               columns =['text','labels'])
train_df.shape

(16000, 2)

In [8]:

# test_dataset = dataset['test']
test_dataset = pd.read_csv('/kaggle/input/punjabi-emotion-data/test_data.csv')
text_test=[]
label_test=[]
for i in range(len(test_dataset)):
    text_test.append(test_dataset.iloc[i]['punjabi_emotion'])
    label_test.append(test_dataset.iloc[i]['labels'])
print(len(text_test),len(label_test))
test_df = pd.DataFrame(list(zip(text_test, label_test)),
               columns =['text','labels'])



2000 2000


In [9]:

validation_dataset = pd.read_csv('/kaggle/input/punjabi-emotion-data/validation_data.csv')
text_validation=[]
label_validation=[]
for i in range(len(validation_dataset)):
    text_validation.append(validation_dataset.iloc[i]['punjabi_emotion'])
    label_validation.append(validation_dataset.iloc[i]['labels'])

print(len(text_validation),len(label_validation))
validation_df = pd.DataFrame(list(zip(text_validation, label_validation)),
               columns =['text','labels'])

# validation.to_csv('english_validation_data.csv')

2000 2000


In [10]:
import re
def strip_punc(txt):
    """
    To get only alphabets a-z and A-Z from text
        :param txt: input text for cleaning [str]
        :return: text which only contains alphabets a-z A-Z [str]
    """
    
    ret = re.findall('[A-Za-z]+',str(txt))
    if ret:
        return True
    else:
        return False

In [11]:
text = train_df['text'].tolist()

kll=[]
for i in text:
    if strip_punc(i):
        g = strip_punc(i)
        kll.append(g)
    else:
        g = strip_punc(i)
        kll.append(g)


train_df['wrong_data_flag'] = kll

train_df.head(10)

train_df = train_df.loc[train_df['wrong_data_flag']!=True]
train_df['wrong_data_flag'].value_counts()

False    15485
Name: wrong_data_flag, dtype: int64

In [12]:


text = validation_df['text'].tolist()

kll=[]
for i in text:
    if strip_punc(i):
        g = strip_punc(i)
        kll.append(g)
    else:
        g = strip_punc(i)
        kll.append(g)


validation_df['wrong_data_flag'] = kll
validation_df = validation_df.loc[validation_df['wrong_data_flag']!=True]
validation_df.shape

(1936, 3)

In [13]:


text = test_df['text'].tolist()

kll=[]
for i in text:
    if strip_punc(i):
        g = strip_punc(i)
        kll.append(g)
    else:
        g = strip_punc(i)
        kll.append(g)


test_df['wrong_data_flag'] = kll
test_df = test_df.loc[test_df['wrong_data_flag']!=True]
test_df.shape

(1942, 3)

In [14]:
test_df.reset_index(drop=True,inplace=True)
validation_df.reset_index(drop=True,inplace=True)
train_df.reset_index(drop=True,inplace=True)

<a id='section03'></a>
### Preparing the Dataset and Dataloader

I will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. I will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing. 
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *SentimentData* Dataset Class
- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the Roberta model for training. 
- I am using the Roberta tokenizer to tokenize the data in the `TITLE` column of the dataframe. 
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/roberta.html#robertatokenizer)
- `target` is the encoded category on the news headline. 
- The *SentimentData* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training. 

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [15]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
# EPOCHS = 1
LEARNING_RATE = 5e-04

from transformers import XLMRobertaTokenizer

# Download the tokenizer for the XLM-Robert `base` model.
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base",truncation=True, do_lower_case=False )
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [16]:
# new_df = train
train_df.shape

(15485, 3)

In [17]:
train_df=train_df[['text','labels']]
validation_df = validation_df[['text','labels']]
test_df = test_df[['text','labels']]

In [18]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [19]:
train_data = train_df
test_data = test_df
validation_data = validation_df

print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))
print("Validation Dataset: {}".format(validation_data.shape))




TRAIN Dataset: (15485, 2)
TEST Dataset: (1942, 2)
Validation Dataset: (1936, 2)


In [20]:
training_set = SentimentData(train_data, tokenizer, MAX_LEN)
validation_set = SentimentData(validation_data,tokenizer,MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

In [21]:
train_data.shape,validation_data.shape,test_data.shape,len(training_set)

((15485, 2), (1936, 2), (1942, 2), 15485)

In [22]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 8
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 8
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `RobertaClass`. 
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs. 
 - The data will be fed to the Roberta Language model as defined in the dataset. 
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction. 
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 
 
#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output. 
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [23]:
from transformers import XLMRobertaForSequenceClassification
# model = RobertaClass()

# sadness (0), joy (1), love (2), anger (3), fear (4), surprise (5).
emotion_map = {0:'sadness',
               1:'joy', 2:'love',3:'anger',4:'fear',5:'surprise'
               }
model  = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=len(emotion_map))
model.to(device)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process. 

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network. 

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size. 
- Subsequent output from the model and the actual category are compared to calculate the loss. 
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a loss of 0.8141926634122427.

In [24]:
!pip install chime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.4 MB/s eta 0:00:0000:0100:010m


In [25]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-6, # args.learning_rate
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from sklearn.metrics import f1_score

## Traning Function

In [26]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model
import chime
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    targets_list=[]
    preds_list =[]
    chime.success()
    for _,data in tqdm(enumerate(training_loader, 0)):
        try:
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids)

            # print(outputs,"\n",outputs.logits)

            loss = loss_function(outputs.logits, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.logits, dim=1)

          # to comment
            # n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            targets_list.extend(list(targets.cpu().detach().numpy()))
            preds_list.extend(list(big_idx.cpu().detach().numpy()))

            # if _%2000==0:
            #     loss_step = tr_loss/nb_tr_steps
            #     accu_step = (n_correct*100)/nb_tr_examples 
            #     print(f"Training Loss per 2000 steps: {loss_step}")
            #     print(f"Training Accuracy per 2000 steps: {accu_step}")

            optimizer.zero_grad()
            loss.backward()
            # # When using GPU
            optimizer.step()
        except:
            continue

    print(len(preds_list),len(targets_list))
    f1_sc = f1_score(targets_list,preds_list,average='micro')

    # print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    print(f'F1 score on Epoch {epoch} : {f1_sc}')
    # epoch_loss = tr_loss/nb_tr_steps
    # epoch_accu = (n_correct*100)/nb_tr_examples
    # print(f"Training Loss Epoch: {epoch_loss}")
    # print(f"Training Accuracy Epoch: {epoch_accu}")

    return f1_sc

In [27]:
import warnings
warnings.filterwarnings('ignore')
EPOCHS = 15
for epoch in range(EPOCHS):
    train(epoch)

968it [03:48,  4.24it/s]


15485 15485
F1 score on Epoch 0 : 0.43797223119147555


968it [03:47,  4.26it/s]


15485 15485
F1 score on Epoch 1 : 0.6093639005489183


968it [03:47,  4.26it/s]


...

15485 15485
F1 score on Epoch 12 : 0.904875686147885



968it [03:47,  4.26it/s]


15485 15485
F1 score on Epoch 13 : 0.9139812721989021


968it [03:46,  4.27it/s]

15485 15485
F1 score on Epoch 14 : 0.9186309331611237


<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data. 

This unseen data is the 20% of `train.tsv` which was seperated during the Dataset creation stage. 
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model. 

As you can see the model is predicting the correct category of a given sample to a 69.47% accuracy which can further be improved by training more.

In [28]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    targets_list=[]
    preds_list =[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(validation_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_function(outputs.logits, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.logits, dim=1)

            # n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            targets_list.extend(list(targets.cpu().detach().numpy()))
            preds_list.extend(list(big_idx.cpu().detach().numpy()))
            
            # if _%5000==0:
            #     loss_step = tr_loss/nb_tr_steps
            #     accu_step = (n_correct*100)/nb_tr_examples
            #     print(f"Validation Loss per 100 steps: {loss_step}")
            #     print(f"Validation Accuracy per 100 steps: {accu_step}")


    # epoch_loss = tr_loss/nb_tr_steps
    # epoch_accu = (n_correct*100)/nb_tr_examples
    # print(f"Validation Loss Epoch: {epoch_loss}")
    # print(f"Validation Accuracy Epoch: {epoch_accu}")
    f1_sc_val =f1_score(targets_list,preds_list,average='micro')
    
    return f1_sc_val


In [29]:
val_ls = []
for i in range(5):
    
    acc = valid(model, validation_loader)
    val_ls.append(acc)
    
final_f1 = sum(val_ls)/len(val_ls)
print("F1 on test data = %0.2f%%" % acc)

242it [00:08, 27.01it/s]
242it [00:08, 27.02it/s]
242it [00:09, 26.21it/s]
242it [00:08, 27.05it/s]
242it [00:08, 27.06it/s]

F1 on test data = 0.82%


## Prediction Time

In [1]:
print('sahib')

sahib


In [31]:
predicted_emotion_ls =[]
predicted_class_id_ls =[]

for i in tqdm(range(len(test_df))):
    test_text = test_df.iloc[i]['text']
    inputs = tokenizer(test_text, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        predicted_emotion = emotion_map[predicted_class_id]
        predicted_emotion_ls.append(predicted_emotion)
        predicted_class_id_ls.append(predicted_class_id)
        
        
test_df['prediction'] = predicted_class_id_ls
test_df['predicted_emotion'] = predicted_emotion_ls
 


100%|██████████| 1942/1942 [00:19<00:00, 101.56it/s]


In [32]:
test_df['true_emotion'] = test_df['labels'].map(emotion_map)
test_df

,text,labels,prediction,predicted_emotion,true_emotion
0,ਮੈਂ ਮਹਿਸੂਸ ਕਰਦਾ ਹਾਂ ਕਿ ਉਹ ਇਸ ਦੇਸ਼ ਦੇ ਨਾਗਰਿਕਾਂ ...,1,1,joy,joy
1,ਮੈਂ ਅਜੇ ਨਹੀਂ ਕਹਾਂਗਾ ਅਤੇ ਮੈਂ ਆਪਣੇ ਆਪ ਨੂੰ ਉੱਤਮ ਮ...,1,1,joy,joy
2,ਮੈਂ ਕਿਸੇ ਵੀ ਵਿਅਕਤੀ ਲਈ ਬਹੁਤ ਕੋਮਲ ਮਹਿਸੂਸ ਕਰਦਾ ਹਾ...,2,2,love,love
3,ਮੈਂ ਉਨ੍ਹਾਂ ਨੂੰ ਗਲੇ ਲਗਾਉਣ ਲਈ ਬੈਠ ਗਿਆ ਅਤੇ ਮਹਿਸੂਸ...,0,0,sadness,sadness
4,ਮੈਂ ਆਪਣੇ ਆਪ ਤੋਂ ਬਹੁਤ ਅਪਮਾਨਿਤ ਮਹਿਸੂਸ ਕਰਦਾ ਹਾਂ,0,3,anger,sadness
...,...,...,...,...,...
1937,ਮੈਂ ਸਿਨਾਈ ਦੇ ਨਫ਼ਰਤ ਵਾਲੇ ਇਜ਼ਰਾਈਲੀ ਕਬਜ਼ੇ ਲਈ ਇੱਕ ...,5,5,surprise,surprise
1938,ਜਦੋਂ ਘਰ ਸਾਫ਼ ਹੁੰਦਾ ਹੈ ਤਾਂ ਖਾਣਾ ਵਧੀਆ ਹੁੰਦਾ ਹੈ ਅ...,0,0,sadness,sadness
1939,ਮੈਂ ਮਹਿਸੂਸ ਕੀਤਾ ਕਿ ਮੇਰਾ ਜਨਮਦਿਨ ਖਾਸ ਮਹਿਸੂਸ ਕਰਨ ...,1,1,joy,joy
1940,ਮੈਂ ਦੁਬਾਰਾ ਉਦਾਸ ਮਹਿਸੂਸ ਕਰਦਾ ਹਾਂ,0,0,sadness,sadness


In [36]:

inputs = tokenizer("ਮੈਂ ਚੰਗਾ ਨਹੀਂ ਅੱਜ ਹਰ ਕੋਈ ਮੈਨੂੰ ਬੁਰਾ ਮਹਿਸੂਸ ਕਰਾ ਰਿਹਾ ਹੈ", return_tensors="pt").to(device)
with torch.no_grad():
    logits = model(**inputs).logits
predicted_class_id = logits.argmax().item()
emotion_map[predicted_class_id]

'sadness'

## Saved Model

In [24]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


In [25]:
test.to_csv('./test_output.csv')